# ProWave - WaveNet-based Protein Generation

Authors: Hans Jakob Damsgaard & Lucas Balling

02456 Deep Learning project: ProGen

## Downloading Tape ##

Run the commmand below if you have not yet installed the [TAPE project](https://github.com/songlab-cal/tape).

In [1]:
#!pip install tape_proteins

#### Importing needed packages

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import tape

#### Import the data

We were unable to make the data download script, `download_data.sh`, run from Jupyter, so instead we ran it manually and simply placed the resulting files in the right folder for TAPE to find them. We import only the natural language processing, unsupervised dataset.

In [ ]:
from tape.datasets import LanguageModelingDataset

# Data stored under `<data-path>/data`
data_path = 'FILL IN HERE'
train_data   = LanguageModelingDataset(data_path, 'train')
valid_data   = LanguageModelingDataset(data_path, 'valid')
holdout_data = LanguageModelingDataset(data_path, 'holdout')

#### Understanding data features

To get a good understanding of the data provided in the imported dataset, we provide plots of certain features and their ranges. Data is already split into the three required subsets; train, validation, and holdout by TAPE, so it is also interesting to understand this split.

In [2]:
raise NotImplementedError('we need to do something here')

NotImplementedError: we need to do something here